In [1]:
from easyquery import Query
from astropy.table import Table, Column, vstack
from astropy.coordinates import SkyCoord, match_coordinates_sky

import SAGA
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA.utils import fill_values_by_query
print(SAGA.__version__)

0.6.1


In [2]:
saga_database = SAGA.Database('/home/yymao/Dropbox/Academia/Collaborations/SAGA', '/home/yymao/Documents/Research/SAGA')

saga_host_catalogs = SAGA.HostCatalog(saga_database)
saga_targets = SAGA.TargetSelection(saga_database, gmm_parameters='gmm_parameters_no_outlier', version=1)

In [3]:
mmt18_hosts = saga_host_catalogs.resolve_id('mmt_2018', 'NSA')

## print target numbers

In [4]:
score_bins = [150, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
d = np.array([np.searchsorted(targets['TARGETING_SCORE'], score_bins) for targets in saga_targets.build_target_catalogs('mmt_2018', return_as='iter')])

/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/saga/SAGA/targets/assign_targeting_score.py:29: RuntimeWarning: overflow encountered in exp
  return np.where(x > model_parameters[2], np.minimum(np.exp((x-model_parameters[3])*model_parameters[4]), model_parameters[5]), 0.0)


In [5]:
print(SAGA.targets.assign_targeting_score.__doc__)

t = Table(d[:,1:] - d[:,:-1], names=score_bins[:-1])
t.add_column(Column(mmt18_hosts, 'NSAID'), 0)
print("# in each bin")
t.pprint(-1, -1)

print()

t = Table(d[:,1:-1], names=['<{}'.format(s) for s in score_bins[1:-1]])
t.add_column(Column(mmt18_hosts, 'NSAID'), 0)
print("# cumulative")
t.pprint(-1, -1)


    Last updated: 05/07/2018
     100 Human selection and Special targets
     150 satellites
     200 within host,  r < 17.77, gri cuts
     300 within host,  r < 20.75, high p_GMM or GMM outliers, gri cuts
     400 within host,  r < 20.75, high-proirity + gri cuts
     500 outwith host, r < 17.77, gri cuts
     600 within host,  r < 20.75, gri cuts, random selection of 50
     700 very high p_GMM, gri cuts
     800 within host,  r < 20.75, gri cuts, everything else
     900 outwith host, r < 20.75, gri cuts
    1000 everything else
    1100 Not in gri/fibermag_r_cut
    1200 Not galaxy
    1300 Not clean
    1400 Has spec but not a satellite
    
# in each bin
NSAID  150 200 300 400 500 600 700 800  900 
------ --- --- --- --- --- --- --- ---- ----
161174   4   1  37 100  11  50  41  218 2763
 52773   2   2  89 150  14  50  28  435 2981
163956   5   5  70 111  12  50  27  493 2729
 69028   4   7 120 248  19  50  35  809 2602
144953   3   3  69 171  23  50  16  566 2591
165082   9  1

## add des to AnaK

In [14]:
def des_cuts(d, host_ra, host_dec, host_dist):
    sep = SkyCoord(d['RA'], d['DEC'], unit='deg').separation(SkyCoord(host_ra, host_dec, unit='deg'))
    d['RHOST_ARCM'] = sep.arcmin
    d['RHOST_KPC'] = (np.sin(sep.radian) * (1000.0 * host_dist))
    
    d['gr'] = d['g_mag'] - d['r_mag']
    d['ri'] = d['r_mag'] - d['i_mag']
    d['gr_err'] = np.hypot(d['g_err'], d['r_err'])
    d['ri_err'] = np.hypot(d['r_err'], d['i_err'])    
    
    q = Query('r_mag < 21',
              'wavg_extended_coadd_i >= 2',
              'gr - gr_err*2.0 < 0.85',
              'ri - ri_err*2.0 < 0.55',
              'RHOST_ARCM < 60',
              'RHOST_KPC < 300')
    
    return q.filter(d)

def match_catalogs(d1, d2, sep_limit=2.0):
    idx, sep, _ = match_coordinates_sky(SkyCoord(d1['RA'], d1['DEC'], unit='deg'), SkyCoord(d2['RA'], d2['DEC'], unit='deg'))
    k = np.where(sep.arcsec < sep_limit)[0]
    return k, idx[k]

def add_des_row_to_target_catalog(host_id, target_catlog, des_to_add):
    des_colnames_lower = [c.lower() for c in des_to_add.colnames]
    to_add = dict()
    out_temp = list()
    for i, (col, type_name) in enumerate(target_catlog.dtype.descr):
        if col.lower() in des_colnames_lower:
            des_col = des_to_add.colnames[des_colnames_lower.index(col.lower())]
            to_add[i] = des_col
            out_temp.append(None)
        elif col == 'OBJID':
            to_add[i] = 'ID'
            out_temp.append(None)
        elif col == 'TARGETING_SCORE':
            out_temp.append(801)
        elif col == 'HOST_NSAID':
            out_temp.append(int(str(host_id).replace('nsa', '')))
        elif col == 'PHOTPTYPE':
            out_temp.append(3)
        elif col == 'has_good_des':
            out_temp.append(True)
        elif type_name[1] in 'i':
            out_temp.append(-1)
        elif type_name[1] in 'f':
            out_temp.append(np.nan)
        elif type_name[1] in 'U':
            out_temp.append('')
        else:
            raise ValueError()
            
    for row in des_to_add:
        out = list(out_temp)
        for i, col in to_add.items():
            out[i] = row[col]
        target_catlog.add_row(tuple(out))
    
    return target_catlog

def modify_target_catalog_with_des(host_id):
    des = saga_database['des', host_id].read()
    des.rename_column('ra', 'RA')
    des.rename_column('dec', 'DEC')
    des = des_cuts(des, *saga_host_catalogs.load_single(host_id)[['RA', 'Dec', 'distance']])
    
    saga_targets.build_target_catalogs(host_id, reload_base=True, recalculate_score=True)
    base_this = saga_targets.target_catalogs[host_id]
    good_sdss_idx = np.where(Query(C.is_clean, C.is_galaxy, C.fibermag_r_cut, C.sat_rcut).mask(base_this))[0]
    
    idx_des, idx_sdss = match_catalogs(des, base_this[good_sdss_idx], 1.0)
    idx_sdss = good_sdss_idx[idx_sdss]
    
    des = des[np.setdiff1d(np.arange(len(des)), idx_des, True)] # not in sdss
    
    base_this['has_good_des'] = False
    base_this['has_good_des'][idx_sdss] = True
    
    need_target = Query(~C.has_spec, 'TARGETING_SCORE >= 900', 'has_good_des')
    print('in sdss, need to target =', need_target.count(base_this))
    print('not in sdss             =', len(des))
    
    fill_values_by_query(base_this, need_target, {'TARGETING_SCORE': 805})
    saga_targets.target_catalogs[host_id] = add_des_row_to_target_catalog(host_id, base_this, des)

In [15]:
host_id = saga_host_catalogs.resolve_id('AnaK', 'string')[0]
modify_target_catalog_with_des(host_id)

in sdss, need to target = 102
not in sdss             = 38


## generate MMT catalogs

In [16]:
for host_id, target_catalog in saga_targets.build_target_catalogs([61945, 165980], return_as='dict').items():
    print('Working host', host_id)
    SAGA.targets.prepare_mmt_catalog(target_catalog, '/home/yymao/Downloads/mmt_{}.cat'.format(host_id))
    print()

Working host nsa61945
# of guide stars     = 289
# of flux stars      = 124
# of rank>1 targets  = 250
# of rank=1 targets = 124
# of rank=2 targets = 6
# of rank=3 targets = 13
# of rank=4 targets = 12
# of rank=5 targets = 16
# of rank=6 targets = 40
# of rank=7 targets = 23
# of rank=8 targets = 140
Writing to /home/yymao/Downloads/mmt_nsa61945.cat

Working host nsa165980
# of guide stars     = 379
# of flux stars      = 147
# of rank>1 targets  = 772
# of rank=1 targets = 147
# of rank=2 targets = 2
# of rank=3 targets = 46
# of rank=4 targets = 119
# of rank=5 targets = 17
# of rank=6 targets = 50
# of rank=7 targets = 22
# of rank=8 targets = 516
Writing to /home/yymao/Downloads/mmt_nsa165980.cat



In [17]:
t = Query('TARGETING_SCORE >= 800', 'TARGETING_SCORE < 900').filter(saga_targets.target_catalogs['nsa61945'])['OBJID', 'RA', 'DEC', 'r_mag']
t['RA'].format = '%.7f'
t['DEC'].format = '%.7f'
t['r_mag'].format = '%.2f'
t.pprint(-1, -1)

       OBJID             RA        DEC     r_mag
------------------- ----------- ---------- -----
1237663784194999070 354.5617043  0.1203141 22.50
1237663784194605375 353.6869377  0.1928568 21.75
1237663784194933360 354.3664335  0.0504893 21.15
1237663462608666769 353.8821706  0.6957298 21.13
1237666408440004671 354.4172913  0.2797575 21.90
1237666408439808570 354.0405575  0.2899991 21.67
1237657190902595936 353.9763469 -0.1326168 20.95
1237666408440005654 354.5528487  0.2788949 20.91
1237663784194671166 353.7572342  0.0703169 20.80
1237657190902530658 353.9269184 -0.1423837 20.97
1237663277925008110 353.7059996  0.4241394 21.06
1237666408439874175 354.1683179  0.3694178 21.00
1237657190902596035 354.0088374 -0.1379302 20.93
1237663784194802323 354.0879743  0.1087812 21.01
1237657190902661744 354.2134146 -0.0182403 20.94
1237663277925335492 354.4274752  0.4325182 21.30
1237663784194736536 353.8919268  0.1822404 20.97
1237663784194999003 354.5479655  0.0663176 20.85
1237657190902596158 